In [ ]:
import os
import hjson
import folium
import glob

# Area visualization

In [ ]:
#Calculate distance from latlon
# https://qiita.com/s-wakaba/items/e12f2a575b6885579df7

from math import sin, cos, acos, radians
earth_rad = 6378.137

def latlng_to_xyz(lat, lng):
    rlat, rlng = radians(lat), radians(lng)
    coslat = cos(rlat)
    return coslat*cos(rlng), coslat*sin(rlng), sin(rlat)

def dist_on_sphere(pos0, pos1, radious=earth_rad):
    xyz0, xyz1 = latlng_to_xyz(*pos0), latlng_to_xyz(*pos1)
    return acos(sum(x * y for x, y in zip(xyz0, xyz1)))*radious

Osaka = 34.702113, 135.494807
Tokyo = 35.681541, 139.767103
London = 51.476853, 0.0

print(dist_on_sphere(Osaka, Tokyo)) # 403.63km
print(dist_on_sphere(London, Tokyo)) # 9571.22km

In [ ]:
#init
map = folium.Map(location=[35.170833333333334, 138.18125], zoom_start=4)

#load
with open('dataset.hjson', 'r') as f:
    dataset = hjson.load(f)

#save map
for pref, v in dataset.items():
    lat0 = v['lat0']
    lat1 = v['lat1']
    lon0 = v['lon0']
    lon1 = v['lon1']
    
    if pref[-1] == '2':
        weight, color = 2, 'red'
    else:
        weight, color = 3, 'black'
        
    #distance
    dist_lat = int(dist_on_sphere([lat0, lon0], [lat1, lon0]))
    dist_lon = int(dist_on_sphere([lat0, lon0], [lat0, lon1]))
    
    #pop-up
    popup = f'{pref} distance_lat={dist_lat} dist_lon={dist_lon}'
    
    #polyline
    folium.PolyLine(
        ([lat0, lon0], [lat0, lon1], [lat1, lon1], [lat1, lon0], [lat0, lon0]),
        popup=popup,
        weight=weight,
        color=color).add_to(map)
    
    #circle marker
    folium.CircleMarker(
        location=[lat0, lon0],
        radius=1,
        color='#3186cc', fill_color='#3186cc',
    ).add_to(map)

map.save(f'/tmp/map.html')
map

# No data on the sea?

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

def value_to_hex(value, vmin=0, vmax=25):
    cmap = cm.plasma
    norm = Normalize(vmin=vmin, vmax=vmax)
    rgb = cmap(norm(value))[:3]
    r, g, b = (np.array(rgb) * 255).astype(int)
    hex = "#{0:02x}{1:02x}{2:02x}".format(r, g, b)
    return hex

areas = ('hokkaido', 'tohoku', 'kanto-hokuriku', 'kyushu', 'nishi-nihon')
year = 1980

map = folium.Map(location=[35.1708333, 138.18125], zoom_start=5)

for area in areas:
    path = os.path.join('meshdata', area, str(year), '*.csv')
    csvpaths = glob.glob(path)
    csvpaths.sort()

    for i, csvpath in enumerate(csvpaths):

        if i % 20 != 0:
            continue

        csv = pd.read_csv(csvpath, comment='#', index_col='date')
        lat = csv.lat[0]
        lon = csv.lon[0]
        null_ratio = (1 - pd.isnull(csv.T2M).mean())   # more null, larger circle
        mean_temp = csv.T2M.mean()
        popup = str(int(null_ratio*100)) + '% ' + str(int(mean_temp)) + '℃'
        folium.CircleMarker([lat, lon], radius=null_ratio*5, popup=popup,
                    fill=True, fill_color=value_to_hex(mean_temp), fill_opacity=0.8).add_to(map)
map